Tech Project 2 - Comparing Ensemble Methods - Stacking Regressor

Diogo Pessoa


In [ ]:
# data_file = 'combined_data.csv'  # Load variable so data_loader can locale the csv filesystem path accordingly.
# %run data_loader.ipynb 

In [ ]:
# %run FeatureEngineering/LassoCV.ipynb

In [ ]:
import time
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor, BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor

base_models = [
    ('knn', KNeighborsRegressor(n_neighbors=20, metric='euclidean')),
    ('gbr', GradientBoostingRegressor(n_estimators=200, learning_rate=0.2))
]

# Define the meta-model
meta_model = BaggingRegressor(n_estimators=200, random_state=42)

# Create the stacking model
stacked_model = StackingRegressor(estimators=base_models, final_estimator=meta_model)


In [ ]:
start = time.time()

stacked_model.fit(x_train_reduced_sc, y_train)
# Calculate the duration
end_time = time.time()
duration = end_time - start_time

# Predict 
pred = stacked_model.predict(x_test_reduced_sc)

mse = mean_squared_error(y_test, pred)
r_two_score = r2_score(y_test, pred)
ex_variance_score = explained_variance_score(y_test, pred)

print(f"{stacked_model.__class__} training took {duration:.4f} seconds.")
print(f'Mean Squared Error on Test Set: {mse:.4f}')
print(f'r2 score: {r_two_score:.4f}')
print(f'explained variance score: {ex_variance_score:.4f}')